In [15]:
# Design connect-4 environment
class Connect_4:
    def __init__(self):
        self.initial_grid = [
            [" ", " ", " "," "," "," "," "],
            [" ", " ", " "," "," "," "," "],
            [" ", " ", " "," "," "," "," "],
            [" ", " ", " "," "," "," "," "],
            [" ", " ", " "," "," "," "," "],
            [" ", " ", " "," "," "," "," "],
                            ]
                             
     #________________________________________________________________________________________________________
    def choose_difficulty(self):
        print("Choose difficulty level from 1 - 10:")
       
        while True:
            try:
                choice = int(input("Enter your choice (1-10): "))
                if 1 <= choice <= 10:
                    return choice
                else:
                    print("Please enter a number between 1 and 10!")
            except ValueError:
                print("Please enter a valid number!")
                
    #________________________________________________________________________________________________________
    def display_grid(self, state):
        print()
        for row in range(6):
            for col in range(7):
                print(' ', state[row][col], ' ', sep='', end='')
                if col < 6:
                    print('|', end='')
            print()
            if row < 5:
                print('---+---+---+---+---+---+---', end='')
                print()
        print()

    #________________________________________________________________________________________________________
    def take_action(self, current_state, action):
        new_state = [
            [" ", " ", " ", " ", " ", " ", " "],
            [" ", " ", " ", " ", " ", " ", " "],
            [" ", " ", " ", " ", " ", " ", " "],
            [" ", " ", " ", " ", " ", " ", " "],
            [" ", " ", " ", " ", " ", " ", " "],
            [" ", " ", " ", " ", " ", " ", " "],
        ]
    
        for row in range(6):
            for col in range(7):
                new_state[row][col] = current_state[row][col]
    
        player, column = action
    
        row = -1
        for r in range(5, -1, -1):  
            if new_state[r][column] == " ":
                row = r
                break
    
        if row != -1:
            new_state[row][column] = player
    
        return new_state

    #________________________________________________________________________________________________________
    def current_player(self, state):
        # Count the number of Xs and Os
        count_Red = 0
        count_Yellow = 0
        for row in range(6):
            for col in range(7):
                symbol = state[row][col]
                if symbol == 'R':
                    count_Red += 1
                elif symbol == 'Y':
                    count_Yellow += 1
        # If they are the same, it's player X's turn
        if count_Red == count_Yellow:
            return 'R'
        # Otherwise, it's player O's turn
        return 'Y'


    #________________________________________________________________________________________________________
    def check_terminal(self, current_state):
        terminal = False
        full = False
        player = None
    
        for row in range(6):
            for col in range(4):
                if current_state[row][col] == current_state[row][col + 1] == current_state[row][col + 2] == current_state[row][col + 3] and current_state[row][col] != " ":
                    terminal = True
                    player = current_state[row][col]
    
        for col in range(7):
            for row in range(3):
                if current_state[row][col] == current_state[row + 1][col] == current_state[row + 2][col] == current_state[row + 3][col] and current_state[row][col] != " ":
                    terminal = True
                    player = current_state[row][col]
    
        for row in range(3):
            for col in range(4):
                if current_state[row][col] == current_state[row + 1][col + 1] == current_state[row + 2][col + 2] == current_state[row + 3][col + 3] and current_state[row][col] != " ":
                    terminal = True
                    player = current_state[row][col]
                if current_state[row + 3][col] == current_state[row + 2][col + 1] == current_state[row + 1][col + 2] == current_state[row][col + 3] and current_state[row + 3][col] != " ":
                    terminal = True
                    player = current_state[row + 3][col]
    
        empty_count = 0
        for row in range(6):
            for col in range(7):
                if current_state[row][col] == " ":
                    empty_count += 1
    
        if empty_count == 0:
            full = True
    
        if terminal:
            if player == "R":
                return 1
            elif player == "Y":
                return -1
        elif full:
            return 0
        else:
            return "Not terminal"

    #________________________________________________________________________________________________________
    
    def available_actions(self, current_state):
        actions = []
        player = self.current_player(current_state)
        for row in range(6):
            for col in range(7):
                if current_state[0][col] == " ":
                    actions.append((player, col))

        return actions

    #________________________________________________________________________________________________________
    def ordered_actions(self, current_state):
        """ Return actions with preference to center columns for better pruning """
        player = self.current_player(current_state)
        center_priority = [3, 2, 4, 1, 5, 0, 6]
        return [(player, col) for col in center_priority if current_state[0][col] == " "]

    #________________________________________________________________________________________________________
    def alpha_beta(self, current_state, depth=4, alpha=float('-inf'), beta=float('inf')):
        result = self.check_terminal(current_state)

        if result != "Not terminal" or depth == 0:
            return (result if result != "Not terminal" else 0), None

        player = self.current_player(current_state)
        best_action = None

        if player == "R":
            max_eval = float('-inf')
            for action in self.ordered_actions(current_state):
                next_state = self.take_action(current_state, action)
                eval, _ = self.alpha_beta(next_state, depth - 1, alpha, beta)
                if eval > max_eval:
                    max_eval = eval
                    best_action = action
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval, best_action
        else:
            min_eval = float('inf')
            for action in self.ordered_actions(current_state):
                next_state = self.take_action(current_state, action)
                eval, _ = self.alpha_beta(next_state, depth - 1, alpha, beta)
                if eval < min_eval:
                    min_eval = eval
                    best_action = action
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval, best_action

    #________________________________________________________________________________________________________
    def human_play(self, current_state):
        self.display_grid(current_state)
        player = self.current_player(current_state)
        print(f"Your turn, you are playing with {player}")

        while True:
            try:
                column = int(input("Choose your action (column 0-6): "))
                if column < 0 or column > 6:
                    print("Column must be between 0 and 6!")
                    continue
                if current_state[0][column] != " ":
                    print("Column is full! Choose another column.")
                    continue
                break
            except ValueError:
                print("Please enter a number between 0 and 6!")

        action = (player, column)
        new_state = self.take_action(current_state, action)
        self.display_grid(new_state)
        return new_state

    #________________________________________________________________________________________________________
    def computer_play(self, current_state, difficulty):
        import random

        player = self.current_player(current_state)
        print(f"Computer's turn, playing as {player}")

        smart_move_chance = random.randint(1, 10)

        if smart_move_chance <= difficulty:
            print("Computer is thinking strategically...")
            _, action = self.alpha_beta(current_state, depth=4)
            if action is None:
                print("No strategic move found, falling back to random.")
                available = self.available_actions(current_state)
                action = random.choice(available)
        else:
            print("Computer is playing randomly...")
            available = self.available_actions(current_state)
            action = random.choice(available)

        new_state = self.take_action(current_state, action)
        print(f"Computer played in column {action[1]}")
        self.display_grid(new_state)
        return new_state
#________________________________________________________________________________________________________
def player_VS_player(self):
    
        
        my_grid = my_game.initial_grid.copy()
        while my_game.check_terminal(my_grid) == "Not terminal":
            print("________________________________________________")
            my_grid = my_game.human_play(my_grid)
            if my_game.check_terminal(my_grid) != "Not terminal":
                if my_game.check_terminal(my_grid) == 1:
                    print("player \"R\"  won !")
                    break
                elif my_game.check_terminal(my_grid) == -1:
                    print("player \"Y\" won !")
                    break
                else:
                    print("Draw !")
                    break
            
            my_grid = my_game.human_play(my_grid)
            if my_game.check_terminal(my_grid) != "Not terminal":
                if my_game.check_terminal(my_grid) == 1:
                    print("player \"R\"  won !")
                    break
                elif my_game.check_terminal(my_grid) == -1:
                    print("player \"Y\" won !")
                    break
                else:
                    print("Draw !")
                    break
#________________________________________________________________________________________
## Testing the game
##my_game = Connect_4()
##my_grid = my_game.initial_grid.copy()
##difficulty = my_game.choose_difficulty()
##while my_game.check_terminal(my_grid) == "Not terminal":
    ##print("________________________________________________")
    ##my_grid = my_game.human_play(my_grid)
    ##if my_game.check_terminal(my_grid) != "Not terminal":
        ##if my_game.check_terminal(my_grid) == 1:
            ##print("You won !")
            ##break
        ##elif my_game.check_terminal(my_grid) == -1:
            ##print("You lost !")
            ##break
        ##else:
            ##print("Draw !")
            ##break
    
    ##my_grid = my_game.computer_play(my_grid, difficulty=difficulty)
    ##if my_game.check_terminal(my_grid) != "Not terminal":
        ##if my_game.check_terminal(my_grid) == 1:
            ##print("You won !")
            ##break
        ##elif my_game.check_terminal(my_grid) == -1:
            ##print("You lost !")
            ##break
        ##else:
            ##print("Draw !")
            ##break

import tkinter as tk
from tkinter import messagebox


class Connect4GUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Connect 4")
        self.root.configure(bg="#1e2b3a")

        self.CELL_SIZE = 90
        self.ROWS = 6
        self.COLS = 7
        self.TOP_PADDING = self.CELL_SIZE

        self.red_score = 0
        self.yellow_score = 0

        self.game = Connect_4()
        self.state = [row[:] for row in self.game.initial_grid]
        self.difficulty = 5
        self.vs_computer = True

        self.create_mode_selection()

    def create_mode_selection(self):
        self.selection_frame = tk.Frame(self.root, bg="#1e2b3a")
        self.selection_frame.pack(pady=100)

        label = tk.Label(self.selection_frame, text="Choose Mode:", font=("Arial", 14, "bold"), fg="white", bg="#1e2b3a")
        label.pack(pady=10)

        self.mode_var = tk.StringVar(value="Player vs Computer")

        for mode in ["Player vs Player", "Player vs Computer"]:
            tk.Radiobutton(self.selection_frame, text=mode, variable=self.mode_var, value=mode,
                           bg="#34495e", fg="white", selectcolor="#1e2b3a", indicatoron=0,
                           font=("Arial", 11), width=20, relief="flat").pack(pady=5)

        tk.Button(self.selection_frame, text="Start Game", command=self.start_selected_mode,
                  bg="#00b894", fg="white", font=("Arial", 12, "bold"), width=20).pack(pady=20)

    def start_selected_mode(self):
        mode = self.mode_var.get()
        self.selection_frame.destroy()
        if mode == "Player vs Player":
            self.vs_computer = False
            self.create_widgets()
            self.reset_game()
        else:
            self.vs_computer = True
            self.create_widgets()
            self.ask_difficulty()

    def create_widgets(self):
        main_frame = tk.Frame(self.root, bg="#1e2b3a")
        main_frame.pack()

        self.left_frame = tk.Frame(main_frame, bg="#1e2b3a", width=100)
        self.left_frame.pack(side=tk.LEFT, padx=20)

        game_frame = tk.Frame(main_frame, bg="#1e2b3a")
        game_frame.pack(side=tk.LEFT)

        self.right_frame = tk.Frame(main_frame, bg="#1e2b3a", width=100)
        self.right_frame.pack(side=tk.LEFT, padx=20)

        self.canvas = tk.Canvas(game_frame, width=self.COLS * self.CELL_SIZE,
                                height=(self.ROWS * self.CELL_SIZE) + self.TOP_PADDING,
                                bg="#3a5fcd", highlightthickness=0)
        self.canvas.pack(pady=20)

        self.hover_circles = [None] * self.COLS

        self.red_box = tk.Label(self.left_frame, text="Player 1\n0 win", fg="white", bg="#1e2b3a",
                                font=("Arial", 12), justify="center")
        self.red_box.pack(pady=10)

        self.yellow_box = tk.Label(self.right_frame, text=("Computer" if self.vs_computer else "Player 2") + "\n0 win",
                                   fg="white", bg="#1e2b3a", font=("Arial", 12), justify="center")
        self.yellow_box.pack(pady=10)

        self.turn_label = tk.Label(self.root, text="", font=("Arial", 14, "bold"), fg="white", bg="#1e2b3a")
        self.turn_label.pack()

        self.restart_btn = tk.Button(self.root, text="Restart Game", command=self.reset_game,
                                     bg="#00b894", fg="white", font=("Arial", 12, "bold"))
        self.restart_btn.pack(pady=10)

        if self.vs_computer:
            self.difficulty_btn = tk.Button(self.root, text="Change Difficulty", command=self.ask_difficulty,
                                            bg="#00b894", fg="white", font=("Arial", 11, "bold"))
            self.difficulty_btn.pack(pady=5)

        self.back_btn = tk.Button(self.root, text="Back to Mode Selection", command= self.back_to_selection,
                                  bg="#d63031", fg="white", font=("Arial", 11, "bold"))
        self.back_btn.pack(pady=5)

        self.canvas.bind("<Button-1>", self.on_click)
        self.canvas.bind("<Motion>", self.on_mouse_move)
        self.canvas.bind("<Leave>", self.on_mouse_leave)

    def back_to_selection(self):
        # Destroy main components if they exist
        if hasattr(self, 'canvas'):
            self.canvas.destroy()
        if hasattr(self, 'restart_btn'):
            self.restart_btn.destroy()
        if hasattr(self, 'difficulty_btn'):
            self.difficulty_btn.destroy()
        if hasattr(self, 'back_btn'):
            self.back_btn.destroy()
        if hasattr(self, 'turn_label'):
            self.turn_label.destroy()
        if hasattr(self, 'left_frame'):
            self.left_frame.destroy()
        if hasattr(self, 'right_frame'):
            self.right_frame.destroy()

        # Clear any remaining widget in the root window
        for widget in self.root.winfo_children():
            widget.destroy()

        # Reset game state and scores
        self.red_score = 0
        self.yellow_score = 0
        self.state = [row[:] for row in self.game.initial_grid]

        # Recreate mode selection UI
        self.create_mode_selection()
    def ask_difficulty(self):
        popup = tk.Toplevel(self.root)
        popup.title("Select Difficulty")
        popup.geometry("300x250")
        popup.configure(bg="#1e2b3a")

        label = tk.Label(popup, text="Choose difficulty:", bg="#1e2b3a", fg="white", font=("Arial", 12, "bold"))
        label.pack(pady=10)

        difficulty_var = tk.StringVar(value="Medium")

        def set_level():
            level = difficulty_var.get()
            if level == "Easy":
                self.difficulty = 1
            elif level == "Medium":
                self.difficulty = 5
            elif level == "Hard":
                self.difficulty = 10
            popup.destroy()
            self.reset_game()

        for text in ["Easy", "Medium", "Hard"]:
            tk.Radiobutton(popup, text=text, variable=difficulty_var, value=text,
                           bg="#34495e", fg="white", selectcolor="#1e2b3a", indicatoron=0,
                           font=("Arial", 11), width=15, relief="flat").pack(pady=5)

        tk.Button(popup, text="Start Game", command=set_level,
                  bg="#00b894", fg="white", font=("Arial", 10, "bold")).pack(pady=15)

    def update_turn_label(self):
        current = self.game.current_player(self.state)
        if current == "R":
            self.turn_label.config(text="Red Player Turn")
        else:
            self.turn_label.config(text=("Computer Turn" if self.vs_computer else "Yellow Player Turn"))

    def draw_board(self):
        self.canvas.delete("all")

        for c in range(self.COLS):
            x1 = c * self.CELL_SIZE + 5
            y1 = 5
            x2 = x1 + self.CELL_SIZE - 10
            y2 = y1 + self.CELL_SIZE - 10
            oval = self.canvas.create_oval(x1, y1, x2, y2, fill="#3a5fcd", outline="white", width=2)
            self.hover_circles[c] = oval

        for r in range(self.ROWS):
            for c in range(self.COLS):
                x1 = c * self.CELL_SIZE + 5
                y1 = self.TOP_PADDING + r * self.CELL_SIZE + 5
                x2 = x1 + self.CELL_SIZE - 10
                y2 = y1 + self.CELL_SIZE - 10
                color = "#0d1b2a"
                if self.state[r][c] == "R":
                    color = "#ff3c38"
                elif self.state[r][c] == "Y":
                    color = "#ffd633"
                self.canvas.create_oval(x1, y1, x2, y2, fill=color, outline="white", width=2)

        self.update_turn_label()

    def on_mouse_move(self, event):
        col = event.x // self.CELL_SIZE
        for i in range(self.COLS):
            self.canvas.itemconfig(self.hover_circles[i], fill="#3a5fcd")

        if 0 <= col < self.COLS:
            current_player = self.game.current_player(self.state)
            color = "#ff3c38" if current_player == "R" else "#ffd633"
            self.canvas.itemconfig(self.hover_circles[col], fill=color)

    def on_mouse_leave(self, event):
        for i in range(self.COLS):
            self.canvas.itemconfig(self.hover_circles[i], fill="#3a5fcd")

    def on_click(self, event):
        col = event.x // self.CELL_SIZE
        current_player = self.game.current_player(self.state)
        action = (current_player, col)
        available = self.game.available_actions(self.state)
        if action not in available:
            return
        self.state = self.game.take_action(self.state, action)
        self.draw_board()
        result = self.game.check_terminal(self.state)
        if result != "Not terminal":
            self.show_result(result)
        elif self.vs_computer:
            self.root.after(500, self.computer_turn)

    def computer_turn(self):
        self.state = self.game.computer_play(self.state, difficulty=self.difficulty)
        self.draw_board()
        result = self.game.check_terminal(self.state)
        if result != "Not terminal":
            self.show_result(result)

    def show_result(self, result):
        if result == 1:
            self.red_score += 1
            self.red_box.config(text="Player 1\n{} win".format(self.red_score))
            messagebox.showinfo("Game End", "Player Red wins!")
        elif result == -1:
            self.yellow_score += 1
            self.yellow_box.config(text=("Computer" if self.vs_computer else "Player 2") + f"\n{self.yellow_score} win")
            messagebox.showinfo("Game Over", ("Computer wins" if self.vs_computer else "Yellow Player wins"))
        else:
            messagebox.showinfo("Game End", "It's a draw!")
        self.canvas.unbind("<Button-1>")

    def reset_game(self):
        self.state = [row[:] for row in self.game.initial_grid]
        self.canvas.bind("<Button-1>", self.on_click)
        self.draw_board()
        self.update_turn_label()

if __name__ == "__main__":
    root = tk.Tk()
    gui = Connect4GUI(root)
    root.mainloop()


Computer's turn, playing as Y
Computer is thinking strategically...
Computer played in column 3

   |   |   |   |   |   |   
---+---+---+---+---+---+---
   |   |   |   |   |   |   
---+---+---+---+---+---+---
   |   |   |   |   |   |   
---+---+---+---+---+---+---
   |   |   |   |   |   |   
---+---+---+---+---+---+---
   |   |   | Y |   |   |   
---+---+---+---+---+---+---
   |   |   | R |   |   |   

Computer's turn, playing as Y
Computer is thinking strategically...
Computer played in column 3

   |   |   |   |   |   |   
---+---+---+---+---+---+---
   |   |   |   |   |   |   
---+---+---+---+---+---+---
   |   |   |   |   |   |   
---+---+---+---+---+---+---
   |   |   | Y |   |   |   
---+---+---+---+---+---+---
   |   |   | Y |   |   |   
---+---+---+---+---+---+---
 R |   |   | R |   |   |   

Computer's turn, playing as Y
Computer is thinking strategically...
Computer played in column 3

   |   |   |   |   |   |   
---+---+---+---+---+---+---
   |   |   | Y |   |   |   
---+---